# Iteration 6: bed utilisation (KPI)

## Plan English Summary

In this iteration we adds another basic performance measure calculation

2. Utilization of critical care beds.

These calculations only use statistics collected in the results collection period.


## Notebook imports

In [1]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [2]:
display(Markdown("inputs/iteration_6a.md"))

modify the code as follows: 

For both unplanned and elective admissions, after a patient has completed treatment, if the simulation time has exceeded the warm-up period, the patient's treatment time should be added to a running total.

Add in a new performance measure that is calculated at the end of the simulation run called "bed utilization". This is calculated as follows:

* The total treatment time divided by (the number of beds multiplied by the results collection period).   This is a percentage.

Add the result to the pandas dataframe.

Show all of the model code: this includes all patient generator functions in the CCUModel class and all code in the Experiment class.

## Date and time of input to LLM

> 2500 3rd December 2023 

## Observations on the LLM Output

* The LLM seems to have correctly added in the collection of total treatment time after a patient treatment has been completed.  This takes account of the warm-up period.  E.g for both unplanned and elective patients we have the follow code inserted:

```python
    if self.env.now > self.experiment.warm_up_period:
        self.experiment.total_treatment_time += treatment_time
```

* When the run of the model is complete the bed utilisation is calculated as follows:

```python
bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
```

* Note that the LLM has included the total treatment time in the `Experiment` class rather than the `CCUModel` like elective cancellations.  This works, but is not consistent.  A second update was run in iteration 6 asking for consistency.
 


## LLM Output 1:

In this modified code, the CCUModel class calculates the total treatment time for both unplanned and elective admissions, and calculates the new performance measure "bed utilization" at the end of the simulation run. If the simulation time has exceeded the warm-up period, the code collects statistics on the performance measures and outputs the results in a pandas dataframe

In [3]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.cancelled_elective_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.cancelled_elective_count += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
            performance_measures = pd.DataFrame({'Cancelled Elective Operations': [self.cancelled_elective_count], 'Bed Utilization': [bed_utilization]})
            print(performance_measures)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()


Patient 1 arrived from the Wards at time 5.6625419722633366
Patient 1 admitted to critical care bed at time 5.6625419722633366
Elective surgery patient 2 arrived at time 17.53126255426438
Elective surgery patient 2 admitted to critical care bed at time 17.53126255426438
Patient 3 arrived from Emergency surgery at time 25.750853170963527
Patient 3 admitted to critical care bed at time 25.750853170963527
Patient 4 arrived from Emergency surgery at time 27.20457575552542
Patient 4 admitted to critical care bed at time 27.20457575552542
Elective surgery patient 5 arrived at time 28.35740596592555
Elective surgery patient 5 admitted to critical care bed at time 28.35740596592555
Patient 5 discharged from critical care bed at time 29.444947371627606
Patient 6 arrived from Emergency surgery at time 32.72353432401366
Patient 6 admitted to critical care bed at time 32.72353432401366
Patient 7 arrived from the Wards at time 34.377235267264616
Patient 7 admitted to critical care bed at time 34.37

## LLM Input 2

In [4]:
display(Markdown("inputs/iteration_6b.md"))

all results collection variables should be stored in the Experiment class.  

Show all of the model code: this includes all patient generator functions in the CCUModel class and all code in the Experiment class.

## LLM Output 2:

Here's the complete code for the CCUModel class and the Experiment class with the new performance measure "bed utilization" calculated at the end of the simulation run and stored in the Experiment class:

In [5]:
import simpy
import numpy as np
import pandas as pd

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 warm_up_period=30*24, 
                 results_collection_period=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.warm_up_period = warm_up_period
        self.results_collection_period = results_collection_period
        self.total_treatment_time = 0
        self.cancelled_elective_count = 0
        self.bed_utilization = 0

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
                if self.env.now > self.experiment.warm_up_period:
                    self.experiment.cancelled_elective_count += 1
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            if self.env.now > self.experiment.warm_up_period:
                self.experiment.total_treatment_time += treatment_time
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.results_collection_period + self.experiment.warm_up_period)
        if self.env.now > self.experiment.warm_up_period:
            bed_utilization = self.experiment.total_treatment_time / (self.experiment.num_critical_care_beds * self.experiment.results_collection_period)
            performance_measures = pd.DataFrame({'Cancelled Elective Operations': [self.experiment.cancelled_elective_count], 'Bed Utilization': [bed_utilization]})
            print(performance_measures)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()


Patient 1 arrived from the Wards at time 0.2527873366781998
Patient 1 admitted to critical care bed at time 0.2527873366781998
Patient 2 arrived from Emergency surgery at time 1.4346652493848968
Patient 2 admitted to critical care bed at time 1.4346652493848968
Patient 3 arrived from Accident and Emergency at time 7.576874066551862
Patient 3 admitted to critical care bed at time 7.576874066551862
Elective surgery patient 4 arrived at time 12.390277011163459
Elective surgery patient 4 admitted to critical care bed at time 12.390277011163459
Patient 4 discharged from critical care bed at time 21.491527895992967
Patient 4 discharged from critical care bed at time 22.38892759120668
Critical care bed is available for next patient at time 26.491527895992967
Critical care bed is available for next patient at time 27.38892759120668
Patient 5 arrived from Accident and Emergency at time 32.03317240021388
Patient 5 admitted to critical care bed at time 32.03317240021388
Elective surgery patient 6

## Modifications to LLM code to test for correct model outputs.

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class. We also update the test of run length splitting into two tests varying a results collection period and warm-up period.

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency." **NEW**: there are 0 cancelled electives.
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different results collection period**: The results collection period of the model is set to 10 hours.
    * Expected result: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than warm-up + result collection period = 720 + 10 = 730 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.
* **Vary warm up period**: Vary the warm-up period while holding the results collection period constant.
    * Expected result: Run length is equal to the `results_collection_period`
    * Actual result **(PASS)**: Run length is equal to the `results_collection_period`

In [6]:
# The extreme value
M = 10_000_000

### Test 1: Extreme value test 1

PASS

In [7]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [8]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number.
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    There are no elective cancellations (as no electives arrive)
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [9]:
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 37.6172409737366
Patient 1 admitted to critical care bed at time 37.6172409737366
Patient 2 arrived from Accident and Emergency at time 51.400323358909944
Patient 2 admitted to critical care bed at time 51.400323358909944
Patient 3 arrived from Accident and Emergency at time 52.79604733527648
Patient 3 admitted to critical care bed at time 52.79604733527648
Patient 3 discharged from critical care bed at time 62.56036811827778
Critical care bed is available for next patient at time 67.56036811827778
Patient 4 arrived from Accident and Emergency at time 82.6189441862719
Patient 4 admitted to critical care bed at time 82.6189441862719
Patient 4 discharged from critical care bed at time 124.98542351884086
Critical care bed is available for next patient at time 129.98542351884086
Patient 5 arrived from Accident and Emergency at time 155.3790986003521
Patient 5 admitted to critical care bed at time 155.3790986003521
Patient 6 arrived from

### Test 2: Extreme value test 2

PASS

In [10]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [11]:
extreme_value_test_2()

Elective surgery patient 1 arrived at time 17.10832474529082
Elective surgery patient 1 admitted to critical care bed at time 17.10832474529082
Elective surgery patient 2 arrived at time 32.07750777576908
Elective surgery patient 2 admitted to critical care bed at time 32.07750777576908
Elective surgery patient 3 arrived at time 46.41212636244628
Elective surgery patient 3 admitted to critical care bed at time 46.41212636244628
Elective surgery patient 3 discharged from critical care bed at time 46.92220622860182
Critical care bed is available for next patient at time 51.92220622860182
Elective surgery patient 4 arrived at time 67.29326353351716
Elective surgery patient 4 admitted to critical care bed at time 67.29326353351716
Elective surgery patient 5 arrived at time 86.84515332029578
Elective surgery patient 5 admitted to critical care bed at time 86.84515332029578
Elective surgery patient 5 discharged from critical care bed at time 98.48265534322562
Elective surgery patient 6 arriv

### Test 3: Different run length

PASS

In [12]:
def test_results_collection_period(new_collect_period):
    '''
    Vary the results collection period while holding the 
    warm-up period constant.

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(results_collection_period=new_collect_period)
    model = run_test(experiment)
    return model.env.now

In [13]:
test_results_collection_period(10)

Patient 1 arrived from Emergency surgery at time 2.176351535486235
Patient 1 admitted to critical care bed at time 2.176351535486235
Patient 2 arrived from Accident and Emergency at time 5.017307678467214
Patient 2 admitted to critical care bed at time 5.017307678467214
Patient 3 arrived from Accident and Emergency at time 6.978708892007289
Patient 3 admitted to critical care bed at time 6.978708892007289
Patient 3 discharged from critical care bed at time 11.95100348540198
Elective surgery patient 4 arrived at time 14.501477625674596
Elective surgery patient 4 admitted to critical care bed at time 14.501477625674596
Critical care bed is available for next patient at time 16.95100348540198
Patient 5 arrived from Emergency surgery at time 21.504196337930495
Patient 5 admitted to critical care bed at time 21.504196337930495
Patient 6 arrived from the Wards at time 23.75880514603791
Patient 6 admitted to critical care bed at time 23.75880514603791
Patient 7 arrived from other hospitals at

730

### Test 4: Extreme value test 3

PASS.

In [14]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    Critical care beds set to 1 or parameter
    
    Expected result: when critical_care_beds=1 queues form after first arrival.
    Cancellations also begin after 1st arrival.

    Note: reported utilisation will not quite be 100% because of the patient still
    in service.
    
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)
    print(f'{experiment.total_treatment_time=}')
    print(f'{experiment.results_collection_period=}')

In [15]:
extreme_value_test_3()

Patient 1 arrived from Emergency surgery at time 0.9121307777426195
Patient 1 admitted to critical care bed at time 0.9121307777426195
Patient 2 arrived from Accident and Emergency at time 4.4358779284504966
Patient 3 arrived from Accident and Emergency at time 13.05329300445024
Patient 4 arrived from Accident and Emergency at time 13.085787260462576
Elective surgery patient 5 arrived at time 20.476852910117362
Elective surgery for patient 5 cancelled due to no available critical care beds at time 20.476852910117362
Patient 6 arrived from Accident and Emergency at time 27.132063977657022
Patient 7 arrived from Accident and Emergency at time 31.41956658435639
Patient 8 arrived from the Wards at time 32.7728815986803
Patient 8 discharged from critical care bed at time 34.134768504258716
Elective surgery patient 9 arrived at time 34.884446022910126
Elective surgery for patient 9 cancelled due to no available critical care beds at time 34.884446022910126
Critical care bed is available for 

### Test 5: Extreme value test 4

PASS

In [16]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [17]:
extreme_value_test_4()

Patient 1 arrived from Accident and Emergency at time 0.32050381942773964
Patient 1 admitted to critical care bed at time 0.32050381942773964
Patient 2 arrived from other hospitals at time 1.9577413419572098
Patient 2 admitted to critical care bed at time 1.9577413419572098
Patient 3 arrived from the Wards at time 3.0357289498943856
Patient 3 admitted to critical care bed at time 3.0357289498943856
Patient 4 arrived from Accident and Emergency at time 3.6116273815392574
Patient 4 admitted to critical care bed at time 3.6116273815392574
Patient 5 arrived from Accident and Emergency at time 4.43428157401447
Patient 5 admitted to critical care bed at time 4.43428157401447
Patient 6 arrived from Emergency surgery at time 17.03099655938421
Patient 6 admitted to critical care bed at time 17.03099655938421
Patient 7 arrived from Accident and Emergency at time 17.749957017831605
Patient 7 admitted to critical care bed at time 17.749957017831605
Elective surgery patient 8 arrived at time 20.005

### Test 6: Vary Warm-up parameter

PASS

In [18]:
def test_warmup_period(new_warmup_period):
    '''
    Vary the warm-up period while holding the 
    results collection period constant.

    Expected result Run length should be no longer than results_collection_period
    '''
    experiment = Experiment(warm_up_period=new_warmup_period)
    model = run_test(experiment)
    return model.env.now, experiment.results_collection_period

In [19]:
test_warmup_period(0)

Patient 1 arrived from Accident and Emergency at time 0.5943718374089692
Patient 1 admitted to critical care bed at time 0.5943718374089692
Patient 2 arrived from Emergency surgery at time 8.951887028533543
Patient 2 admitted to critical care bed at time 8.951887028533543
Patient 3 arrived from Emergency surgery at time 13.719216788082665
Patient 3 admitted to critical care bed at time 13.719216788082665
Patient 4 arrived from Accident and Emergency at time 14.12823922740447
Patient 4 admitted to critical care bed at time 14.12823922740447
Elective surgery patient 5 arrived at time 19.784887440438705
Elective surgery patient 5 admitted to critical care bed at time 19.784887440438705
Patient 5 discharged from critical care bed at time 21.868919079633173
Patient 6 arrived from Accident and Emergency at time 22.99197112794892
Patient 6 admitted to critical care bed at time 22.99197112794892
Critical care bed is available for next patient at time 26.868919079633173
Patient 7 arrived from E

(8640, 8640)